## Quantization Noise
When samples of a continuous signal are quantized, the difference between the real value $x(nT_s)$ and the quantized value $v(nT_s)$ represents an error.

\begin{equation}
e = x - v
\end{equation}

### Example
Consider the signal $x(t) = A\sin(2\pi t)$ which is to be sampled and the sample values represented via a $b$ bit digital representation.

The signal is to be represented by $L=2^b$ uniformly spaced levels with spacing

\begin{equation}
\Delta = \frac{2A}{L}
\end{equation}

Consider the case where $A=1$, $b=3$ with a midtread quantizer.

The quantization levels are at $-4\Delta,-3\Delta, -2\Delta, -\Delta, 0, \Delta, 2\Delta, 3\Delta$ 

The input levels and corresponding quantization levels are
1. $[-\Delta/2, \Delta/2]\rightarrow 0$ 
1. $[\Delta/2,3/2\Delta]\rightarrow \Delta$
1. $[3\Delta/2,5/2\Delta]\rightarrow 2\Delta$
1. $>5/2\Delta\rightarrow 3\Delta$
1. $[-\Delta/2,-3/2\Delta]\rightarrow -\Delta$
1. $[-3\Delta/2,-5/2\Delta]\rightarrow -2\Delta$
1. $[-5\Delta/2,-7/2\Delta]\rightarrow -3\Delta$
1. $<-7/2\Delta\rightarrow -4\Delta$



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

A = 1
b = 16
delta = (2 * A) / (2 ** b)

I = np.linspace(-A, A, 1000)
O = np.zeros(len(I))

output_levels = []
for indx in range(-2 ** (b - 1), 2 **(b -1)):
    output_levels.append(indx * delta)

input_levels = np.array(output_levels) - delta / 2

In [ ]:
for indx in range(2 ** b - 1):
    O[(I > input_levels[indx]) & (I < input_levels[indx + 1])] = output_levels[indx]
    
O[(I > input_levels[-1])] = output_levels[-1]

def quantizer(input_level, input_levels, output_levels):
    for indx in range(2 ** b - 1):
        if (input_level > input_levels[indx]) & (input_level < input_levels[indx + 1]):
            return output_levels[indx]
    
    if input_level > input_levels[-1]:
        return output_levels[-1]

    

Input output mapping

In [ ]:
plt.plot(I, O)
plt.grid(True)
plt.xlim([-1.1 * A, 1.1 *A])
plt.ylim([-1.1 * A, 1.1 *A]);
plt.title('Input Output Mapping')
plt.xlabel('Input')
plt.ylabel('Output')
plt.savefig('3bit.jpg', dpi=300)

### Binary representation
Using the two's complement
1. $0 \rightarrow 000$
1. $\Delta \rightarrow 001$
1. $...$

In [ ]:
quantizer(0.4, input_levels, output_levels)

In [ ]:
t = np.linspace(-1, 1, 1000)
signal = A * np.sin(2 * np.pi * t)
q_signal = np.zeros(len(signal))

for indx in range(len(signal)):
    q_signal[indx] = quantizer(signal[indx],
                              input_levels,
                              output_levels)

In [ ]:
plt.subplot(211)
plt.plot(t, signal, 'b', linewidth=4)
plt.plot(t, q_signal, 'r', linewidth=4)
plt.grid(True)

error = signal - q_signal
plt.subplot(212)
plt.plot(t, error, 'b', linewidth=4)
plt.grid(True)
plt.savefig('quant.jpg', dpi=300)

In [ ]:
plt.hist(signal - q_signal, 20);

## Error Analysis

For the uniform quantizer, the error is bounded between $-\Delta/2$ and $\Delta/2$.

If $L$ is large enough we can assume that the error is uniformly distributed between $-\Delta/2$ and $\Delta/2$. That is the probability distribution is
$\begin{equation}
f_E(e)=\left\{ \begin{array}{ll}
\frac{1}{\Delta} & \textrm{ $-\frac{\Delta}{2} \leq e \leq \frac{\Delta}{2}$}\\
0 & \textrm{otherwise}
\end{array} \right.
\end{equation}
$

## Question
Compute 
1. $\int_{-\infty}^\infty f_E(e)de$
1. $\int_{-\infty}^\infty ef_E(e)de$
1. $\int_{-\infty}^\infty e^2f_E(e)de$



It can be shown that the variance is 

\begin{equation}
\sigma^2=\frac{\Delta^2}{12}
\end{equation}

Recall that
\begin{equation}
\Delta = \frac{2A}{2^b}
\end{equation}

We can show that 
\begin{equation}
\sigma^2 = \frac{1}{3}A^22^{-2b}
\end{equation}

The SNR is given by 
\begin{equation}
SNR = \frac{P}{\sigma^2}
\end{equation}

For a sinusoidal signal, $P=\frac{A^2}{2}$

We can show that 
\begin{equation}
SNR=\frac{3}{2}2^{2b}
\end{equation}

In dB we have 
\begin{equation}
10\log_{10}(SNR)=1.8 + 6b
\end{equation}

In [ ]:
np.var(error)

In [ ]:
delta ** 2 / 12
